In [ ]:
import pandas as pd

In [ ]:
import numpy  as np

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense

In [ ]:
df_ratings_cmfrec = pd.read_csv('ratings_colab.csv',index_col=0)

In [ ]:
df_ratings_cmfrec.head()

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [ ]:
df_ratings_cmfrec.UserID.min(), df_ratings_cmfrec.UserID.max()

(1, 6040)

In [ ]:
df_ratings_cmfrec.MovieID.min(), df_ratings_cmfrec.MovieID.max()

(1, 3952)

In [ ]:
users = df_ratings_cmfrec.UserID.unique()
movies = df_ratings_cmfrec.MovieID.unique()

userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [ ]:
df_ratings_cmfrec['UserID'] = df_ratings_cmfrec['UserID'].apply(lambda x: userid2idx[x])
df_ratings_cmfrec['MovieID'] = df_ratings_cmfrec['MovieID'].apply(lambda x: movieid2idx[x])
split = np.random.rand(len(df_ratings_cmfrec)) < 0.8
train = df_ratings_cmfrec[split]
valid = df_ratings_cmfrec[~split]
print(train.shape, valid.shape)

(799482, 3) (200727, 3)


In [ ]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(df_ratings_cmfrec, test_size=0.2, random_state=42)

In [ ]:
train.head()

,UserID,MovieID,Rating
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 799482 entries, 0 to 1000208
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   UserID   799482 non-null  int64
 1   MovieID  799482 non-null  int64
 2   Rating   799482 non-null  int64
dtypes: int64(3)
memory usage: 24.4 MB


In [ ]:
df_ratings_cmfrec['UserID'].nunique()

6040

In [ ]:
df_ratings_cmfrec['MovieID'].nunique()

3706

In [ ]:
num_users = df_ratings_cmfrec['UserID'].nunique()
num_movies = df_ratings_cmfrec['MovieID'].nunique()
# total_num_movies= 3952
embedding_size = 8

In [ ]:
user_input = Input(shape=(1,), name='user_input')
movie_input = Input(shape=(1,), name='movie_input')

In [ ]:
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size, name='user_embedding')(user_input)
movie_embedding = Embedding(input_dim=num_movies, output_dim=embedding_size, name='movie_embedding')(movie_input)


In [ ]:
user_vec = Flatten(name='user_flatten')(user_embedding)
movie_vec = Flatten(name='movie_flatten')(movie_embedding)

In [ ]:
concat = Concatenate()([user_vec, movie_vec])

In [ ]:
hidden = Dense(128, activation='relu')(concat)
output = Dense(1, activation='linear')(hidden)

In [ ]:
model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ movie_input (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_embedding            │ (None, 1, 8)           │         48,320 │ user_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ movie_embedding           │ (None, 1, 8)           │         29,648 │ movie_input[0][0]      │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_flatten (Flatten)    │ (None, 8)              │              0 │ user_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ movie_flatten (Flatten)   │ (None, 8)              │              0 │ movie_embedding[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 16)             │              0 │ user_flatten[0][0],    │
│                           │                        │                │ movie_flatten[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │          2,176 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            129 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 80,273 (313.57 KB)

 Trainable params: 80,273 (313.57 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hist = model.fit([train.UserID, train.MovieID], train.Rating, epochs=20,batch_size=64, validation_split=0.1)

Epoch 1/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 1.3683 - val_loss: 0.9971
Epoch 2/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 0.8060 - val_loss: 0.9928
Epoch 3/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 0.7724 - val_loss: 1.0063
Epoch 4/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - loss: 0.7512 - val_loss: 1.0012
Epoch 5/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step - loss: 0.7361 - val_loss: 0.9988
Epoch 6/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 0.7284 - val_loss: 1.0045
Epoch 7/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - loss: 0.7191 - val_loss: 1.0216
Epoch 8/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.7095 - val_loss: 1.0147
Epoch 9/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 0.6981 - val_loss: 1.0238
Epoch 10/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 0.6882 - val_loss: 1.0285
Epoch 11/20
11243/11243 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.6789 - val_loss: 1.02

In [ ]:
y_pred = model.predict([valid.UserID, valid.MovieID], verbose=0)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mape =  mean_absolute_percentage_error(valid.Rating, y_pred)
print('Mean Absolute Percentage Error: {:.3f}'.format(mape))

Mean Absolute Percentage Error: 0.264
